In [1]:
import pandas as pd
import numpy as np
import gensim
import re
import json

In [2]:
df_review = pd.read_pickle('yelp_review_cleaned_0710v3.pkl')

In [3]:
df_review.head(2)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma_clean,sent_count,lemmatized,count_vec
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, love, the, meat, love, the,...",6,[love the staff love the meat love the place p...,"[(a, 3), (and, 1), (around, 1), (ask, 1), (can..."
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...",3,[super simple place but amazing nonetheless PR...,"[(30, 1), (a, 1), (amazing, 1), (and, 3), (aro..."


In [4]:
review1 = df_review[df_review['stars'] == 1]['lemma_clean']
review2 = df_review[df_review['stars'] == 5]['lemma_clean']
review1_2 = pd.concat([review1,review2])
# better to train word2vec in with all data

In [5]:
%time model = gensim.models.KeyedVectors.load_word2vec_format(\
                        '/home/vagrant/word2vec-package/GoogleNews-vectors-negative300.bin', binary=True)


CPU times: user 1min 18s, sys: 4.11 s, total: 1min 22s
Wall time: 1min 23s


In [14]:
# model[model.wv.vocab][:10]
len(model.wv.vocab.keys())

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


3000000

In [ ]:
# how to load the stanford pretrained word2vec
# http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [6]:
model1 = gensim.models.word2vec.Word2Vec(review1_2,
                                        size = 200, 
                                        window = 5, 
                                        min_count = 5,
                                        workers = 4,
                                        hs = 1,
                                        negative = 0)

#model.most_similar('happy')
#model.doesnt_match(['king','queen','prince','Mike'])
#model.similarity('woman', 'man')

In [110]:
# model2 = gensim.models.word2vec.Word2Vec(review1_2,
#                                         size = 200, # bigger data, bigger size, better model
#                                         window = 5, 
#                                         min_count = 5,
#                                         workers = 4, # only matter with Cython
#                                         hs = 1,
#                                         negative = 0)

In [7]:
len(model1['phenomenal']) # raw NumPy vector of a word

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


200

In [8]:
model1.most_similar('service')

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'loyal', 0.48347151279449463),
 (u'relation', 0.4748350977897644),
 (u'satisfaction', 0.4667649567127228),
 (u'workmanship', 0.43191635608673096),
 (u'turnaround', 0.4208248257637024),
 (u'sevice', 0.4205753803253174),
 (u'value', 0.41442200541496277),
 (u'appreciation', 0.38947513699531555),
 (u'waitstaff', 0.38690808415412903),
 (u'elise', 0.376765638589859)]

In [ ]:
# You can store/load models using the standard gensim methods:
# model.save('/tmp/mymodel')
# new_model = gensim.models.Word2Vec.load('/tmp/mymodel')

## Get features from word2vec

In [26]:
zz = zip(model1.wv.index2word, model.wv.syn0)
# model.wv.vocab gives a dictionary of words and their vectors

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [9]:
# zip all the words and their vectors from word2vec
w2v = dict(zip(model1.wv.index2word, model1.wv.syn0))

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [76]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        # import pdb; pdb.set_trace()
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [78]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
# TRY: XGBOOST classifer!!!
etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

etree_w2v_rf = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", RandomForestClassifier(n_estimators=20))])

In [81]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

svc_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", SVC())])


In [67]:
stars1 = df_review[df_review['stars'] == 1]
stars2 = df_review[df_review['stars'] == 5]
stars3 = df_review[df_review['stars'] == 3]
train_size = 0.75
test_size = 1 - train_size

df_train = pd.concat([stars1[:10000],stars2[:10000]])
df_test = pd.concat([stars1[10000:len(stars1)],stars2[10000:len(stars1)]])


#df_train = pd.concat([stars1[:int(len(stars1)*train_size)],stars2[:int(len(stars2)*train_size)]])
#df_test = pd.concat([stars1[int(-len(stars1)*test_size):],stars2[int(-len(stars2)*test_size):]])


In [68]:
y_train = df_train['stars'].apply(lambda x: 0 if x ==1 else 1)
x_train = df_train['lemma_clean']
y_test = df_test['stars'].apply(lambda x: 0 if x ==1 else 1)
x_test = df_test['lemma_clean']
x_test_3 = stars3['lemma_clean']


In [59]:
etree_w2v.fit(x_train.values, y_train.values)
# etree_w2v_tfidf.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7f917c068fd0>), ('extra trees', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impur...imators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [79]:
etree_w2v_rf.fit(x_train.values, y_train.values)
etree_w2v_rf.score(x_train.values, y_train.values)
etree_w2v_rf.score(x_test, y_test)

0.99955

In [82]:
svc_w2v.fit(x_train.values, y_train.values)
print svc_w2v.score(x_train.values, y_train.values)
print svc_w2v.score(x_test, y_test)

0.91195
0.9078665442301805


In [63]:
x_train[:10]

13    [PRON, think, tidy, s, flower, have, a, great,...
17    [PRON, too, have, be, try, to, book, an, appt,...
20    [really, excited, to, hear, of, this, restaura...
24    [food, be, very, bland, not, authentic, at, al...
32    [if, PRON, have, not, yet, try, wasabi, do, no...
44    [terrible, service, and, not, so, great, drink...
45    [bad, customer, service, ever, \n, manager, on...
61    [after, read, the, review, on, yelp, PRON, boy...
62    [come, here, for, a, friend, s, birthday, PRON...
74    [after, read, review, on, this, place, think, ...
Name: lemma_clean, dtype: object

In [61]:
etree_w2v.score(x_train.values, y_train.values)
# check the shape of training data and y

0.99995

In [65]:
# etree_w2v_tfidf.score(x_train.values, y_train.values)

In [66]:
etree_w2v.score(x_test, y_test)

0.8798591980410162

In [69]:
pd.Series(etree_w2v.predict(x_test_3)).value_counts()

1    6986
0    4821
dtype: int64

In [40]:
pd.Series(y_train).value_counts()

5    10000
1    10000
Name: stars, dtype: int64

In [71]:
6986.0/(4821+6986)

0.5916828999745913

In [80]:
y_train.describe()

count    20000.00000
mean         3.00000
std          2.00005
min          1.00000
25%          1.00000
50%          3.00000
75%          5.00000
max          5.00000
Name: stars, dtype: float64